# 용의자 대화 녹음기 (메모리)

- 사용자가 입력한 정보는 LLM에 전달하여 답변을 받아 오고 UI 채팅창을 통해 출력된다.
- 사용자가 **"기록 시작" , "녹음 시작"** 지시를 하면 이후의 대화 내용은 기억하고, 그 이전의 내용은 기억하지 않는다.
- 사용자가 **"기록 중지" , "녹음 중지"** 지시를 하면 그 이후의 내용은 기억하지 않는다.
- 사용자가 **"기록 삭제" , "녹음 삭제"** 지시를 하면 모든 내용은 삭제한다.
- Gradio UI를 통해 녹음기의 상태를 표시한다.
- memory는 현재 세션에서만 캐시 형태로 사용될 수 있도록 관리한다.

**※ 대화 기록이 있는 경우, 기록이 없는 경우에 대화 요약을 요청하여 결과를 비교한다.**

## 환경 설정 및 OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
from dotenv import load_dotenv

from typing import List, TypedDict
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser
from langgraph.graph import StateGraph, END
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.tools import tool
from langchain_teddynote.graphs import visualize_graph

import gradio as gr

# .env 파일 로드
load_dotenv()

# OpenAI LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

## 상태 관리 (메모리)

In [ ]:
class AppState(BaseModel):
    """대화 목록과 기록 상태를 저장하는 모델"""
    is_recording: bool = False
    memory: List[HumanMessage | AIMessage] = Field(description="사용자와 AI의 대화 목록", default_factory=list)

    def get_status(self) -> str:
        """현재 기록 상태를 문자열로 반환"""
        return "🔴 기록 중" if self.is_recording else "⚫️ 기록 대기"

## 메모리 관리 도구 (tool)

In [ ]:
@tool
def start_memory_recording(state: AppState) -> str:
    """사용자가 대화 기록 시작을 요청할 때 이 도구를 호출합니다. '기록 시작', '녹음 시작' 등의 명령어에 해당합니다."""
    state.is_recording = True
    return "지금부터 대화를 기록하기 시작합니다."

@tool
def stop_memory_recording(state: AppState) -> str:
    """사용자가 대화 기록 중지를 요청할 때 이 도구를 호출합니다. '기록 중지', '녹음 중지' 등의 명령어에 해당합니다."""
    state.is_recording = False
    return "지금부터 대화를 기록하지 않습니다."

@tool
def clear_all_memory(state: AppState) -> str:
    """사용자가 모든 대화 기록 삭제를 요청할 때 이 도구를 호출합니다. '기록 삭제', '녹음 삭제' 등의 명령어에 해당합니다."""
    state.memory.clear()
    return "모든 대화 기록을 삭제했습니다."

## 프롬프트 구성
1. `system_prompt`: 프롬프트 템플릿(`ChatPromptTemplate`)에 고정적으로 포함된 부분 - AI의 역할이나 행동 지침을 정의하는 정적인 텍스트로, 대화가 진행되어도 변하지 않음.

2. chat_history (state.memory): `MessagesPlaceholder를` 통해 동적으로 채워지는 대화 기록 - `clear_all_memory` 함수는 바로 이 state.memory 리스트의 내용만 삭제.

* 따라서 '기록 삭제' 명령을 실행하면 state.memory 리스트는 비워지지만, system_prompt는 프롬프트 템플릿 구조의 일부로서 그대로 유지 - 이후 새로운 대화를 시작할 때도 시스템 프롬프트는 항상 LLM에 전달됨.

In [ ]:
system_prompt = """
당신은 명탐정 코난의 수사하는 사건의 유력한 용의자 입니다.
탐정의 질문에 적절하게 답변을 해야 합니다.
탐정은 대화 기록을 관리하는 녹음기를 사용할 수 있습니다.: 'start_memory_recording', 'stop_memory_recording', 'clear_all_memory'.
사용자의 요청을 분석해서 적절한 도구를 사용할 수 있도록 허용해 주어야 합니다.
예를 들어, 사용자가 '녹음을 시작합니다.' 라고 말하면 'start_memory_recording' 도구를 호출해야 합니다.
그 외 모든 일반적인 대화에는 도구를 사용하지 말고 직접 답변해야 하고 탐정에게 추가적인 질문을 해서는 안됩니다..
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

## LCEL 파이프라인 설정

In [ ]:
tools = [start_memory_recording, stop_memory_recording, clear_all_memory]

# 도구 이름을 키로, 함수를 값으로 하는 딕셔너리를 생성
tool_map = {t.name: t for t in tools}

# llm에 도구들을 바인딩
llm_with_tools = llm.bind_tools(tools)

# LCEL 체인을 구성
chain = prompt | llm_with_tools

## UI 챗봇 인터페이스 함수

In [ ]:
def chat_fn(user_message: str, history: list, state: AppState):
    response = chain.invoke({"input": user_message, "chat_history": state.memory})

    ai_message = ""
    # LLM 응답이 도구 호출일 경우
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            if tool_name in tool_map:
                tool_to_call = tool_map[tool_name]
                ai_message = tool_to_call.func(state=state)
    else:
        # 일반 메시지일 경우
        ai_message = response.content

    # history는 UI 표시용, state.memory는 LLM 전달용
    history.append({"role": "user", "content": user_message})
    history.append({"role": "assistant", "content": ai_message})

    # 기록 상태일 때만 실제 메모리에 대화 내용 추가
    if state.is_recording:
        # 도구 호출이 아닌 일반 대화만 메모리에 저장
        if not response.tool_calls:
            state.memory.extend(
                [HumanMessage(content=user_message), AIMessage(content=ai_message)]
            )

    return history, state, state.get_status()

### Gradio UI

In [ ]:
# Gradio UI 레이아웃 설계
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    initial_state = AppState()
    state = gr.State(value=initial_state)

    gr.Markdown("### 🕵️‍♂️ 탐정의 녹음기!")
    gr.Markdown("`기록 시작`, `기록 중지`, `기록 삭제` 명령어를 사용해 보세요.")

    with gr.Row():
        status_indicator = gr.Textbox(
            value=initial_state.get_status(),
            label="메모리 상태",
            interactive=False,
        )

    chatbot = gr.Chatbot(label="대화창", height=300, type="messages")

    txt = gr.Textbox(placeholder="어제 저녁 9시에 어디에 계셨죠?", show_label=False)
    txt.submit(
        chat_fn,
        inputs=[txt, chatbot, state],
        outputs=[chatbot, state, status_indicator]
    )

    # 입력창 초기화
    txt.submit(lambda: "", None, txt)

# 웹 서버 실행
demo.launch()

-----
** End of Documents **